## Importing Libraries

In [1]:
import argparse
import datetime as dte
import os
import json

import data_formatters.base
import expt_settings.configs
import libs.hyperparam_opt
import libs.tft_model
import libs.utils as utils
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf

import warnings 

warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

ExperimentConfig = expt_settings.configs.ExperimentConfig
HyperparamOptManager = libs.hyperparam_opt.HyperparamOptManager
ModelClass = libs.tft_model.TemporalFusionTransformer
tf.experimental.output_all_intermediates(True)

### Experiment Metrics Tracking

In [2]:
with open('volatility_dataset_experiments.json', 'r') as f:
    loss_experiment_tracker = json.load(f)

dataset_name = "volatility"
dataset_folder_path = "volatility_dataset"

### Creating Model

In [3]:
loss_experiment_tracker[dataset_folder_path] = {}

name = dataset_name
output_folder = dataset_folder_path

use_tensorflow_with_gpu = True
print("Using output folder {}".format(output_folder))

config = ExperimentConfig(name, output_folder)
formatter = config.make_data_formatter()

expt_name=name
use_gpu=use_tensorflow_with_gpu
model_folder=os.path.join(config.model_folder, "fixed")
data_csv_path=config.data_csv_path
data_formatter=formatter
use_testing_mode=True

num_repeats = 1

if not isinstance(data_formatter, data_formatters.base.GenericDataFormatter):
    raise ValueError(
        "Data formatters should inherit from" +
        "AbstractDataFormatter! Type={}".format(type(data_formatter)))

# Tensorflow setup
default_keras_session = tf.keras.backend.get_session()

if use_gpu:
    tf_config = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

else:
    tf_config = utils.get_default_tensorflow_config(tf_device="cpu")

print("*** Training from defined parameters for {} ***".format(expt_name))

print("Loading & splitting data...")
raw_data = pd.read_csv(data_csv_path, index_col=0)
train, valid, test = data_formatter.split_data(raw_data)
train_samples, valid_samples = data_formatter.get_num_samples_for_calibration()

# Sets up default params
fixed_params = data_formatter.get_experiment_params()
params = data_formatter.get_default_model_params()
params["model_folder"] = model_folder

# Parameter overrides for testing only! Small sizes used to speed up script.
if use_testing_mode:
    fixed_params["num_epochs"] = 1
    params["hidden_layer_size"] = 5
    train_samples, valid_samples = 100, 10

# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                   fixed_params, model_folder)

# Training -- one iteration only
print("*** Running calibration ***")
print("Params Selected:")

for k in params:
    print("{}: {}".format(k, params[k]))

best_loss = np.Inf

for _ in range(num_repeats):

    tf.reset_default_graph()
    with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

        tf.keras.backend.set_session(sess)

        params = opt_manager.get_next_parameters()
        model = ModelClass(params, use_cudnn=use_gpu)

        if not model.training_data_cached():
            model.cache_batched_data(train, "train", num_samples=train_samples)
            model.cache_batched_data(valid, "valid", num_samples=valid_samples)

        sess.run(tf.global_variables_initializer())
        model.fit()

        val_loss = model.evaluate()

        if val_loss < best_loss:
            opt_manager.update_score(params, val_loss, model)
            best_loss = val_loss

        tf.keras.backend.set_session(default_keras_session)

Using output folder volatility_dataset

Metal device set to: Apple M1 Pro
Selecting GPU ID=0
*** Training from defined parameters for volatility ***
Loading & splitting data...
Formatting train-valid-test splits.
Setting scalers with training data...
*** Loading hyperparm manager ***
*** Running calibration ***
Params Selected:
dropout_rate: 0.3
hidden_layer_size: 5
learning_rate: 0.01
minibatch_size: 64
max_gradient_norm: 0.01
num_heads: 1
stack_size: 1
model_folder: volatility_dataset/saved_models/volatility/fixed
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <Input

                                                                                                  
 add (Add)                      (None, 1)            0           ['dense_3[0][0]',                
                                                                  'multiply[0][0]']               
                                                                                                  
 multiply_1 (Multiply)          (None, 1, 5)         0           ['dense_10[0][0]',               
                                                                  'dense_11[0][0]']               
                                                                                                  
 layer_normalization (LayerNorm  (None, 1)           2           ['add[0][0]']                    
 alization)                                                                                       
                                                                                                  
 add_1 (Ad

 sequential (Sequential)        (None, 257, 5)       35          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_2[0][0]']   
                                                                                                  
 sequential_1 (Sequential)      (None, 257, 5)       155         ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_3[0][0]']   
                                                                                                  
 sequential_2 (Sequential)      (None, 257, 5)       265         ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_4[0][0]']   
                                                                                                  
 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 5)]      0           ['layer_normalization_2[0][0]']  
 sformer/ExpandDims_3 (TensorFl                                                                   
 owOpLayer)                                                                                       
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['time_distributed_4[0][0]',     
 sformer/add_6 (TensorFlowOpLay                                   'time_distributed_5[0][0]']     
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 tf_op_layer_TemporalFusionTran  [(None, 5, 5)]      0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_26 (Tens                                  former/strided_slice_14[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 5, 5)]      0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_27 (Tens                                  former/strided_slice_14[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 time_distributed_6 (TimeDistri  (None, 252, 5)      30          ['activation_7[0][0]']           
 buted)                                                                                           
          

...

 dense_23 (Dense)               (None, 5)            30          ['dropout_4[0][0]']              
                                                                                                  
 dense_26 (Dense)               (None, 5)            30          ['dropout_5[0][0]']              
                                                                                                  
 dense_27 (Dense)               (None, 5)            30          ['dropout_5[0][0]']              
                                                                                                  
 layer_normalization_14 (LayerN  (None, 5, 5)        10          ['add_14[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 add_15 (Add)                   (None, 5, 5)         0           ['tf_op_layer_TemporalFusionTrans
          

 ayer)                                                            'layer_normalization_17[0][0]', 
                                                                  'layer_normalization_18[0][0]', 
                                                                  'layer_normalization_19[0][0]'] 
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['multiply_15[0][0]']            
 sformer/Sum_1 (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 layer_normalization_4 (LayerNo  (None, 5)           10          ['add_4[0][0]']                  
 rmalization)                                                                                     
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 5)]      0           ['layer_normalization_3[0][0]']  
 sformer/ExpandDims_5 (TensorFl                                                                   
 owOpLayer)                                                                                       
                                                                                                  
 time_distributed_65 (TimeDistr  (None, 257, 5)      30          ['layer_normalization_20[0][0]'] 
 ibuted)                                                                                          
                                                                                                  
 time_distributed_66 (TimeDistr  (None, 1, 5)        25          ['tf_op_layer_TemporalFusionTrans
 ibuted)                                                         former/ExpandDims_5[0][0]']      
          

 yer)                                                                                             
                                                                                                  
 lambda (Lambda)                (None, 257, 257)     0           ['dense_96[0][0]',               
                                                                  'dense_97[0][0]']               
                                                                                                  
 lambda_1 (Lambda)              (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_22 (Add)                   (None, 257, 257)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
          

 tf_op_layer_TemporalFusionTran  [(None, 5, 5)]      0           ['layer_normalization_24[0][0]'] 
 sformer/strided_slice_34 (Tens                                                                   
 orFlowOpLayer)                                                                                   
                                                                                                  
 time_distributed_78 (TimeDistr  (None, 5, 3)        18          ['tf_op_layer_TemporalFusionTrans
 ibuted)                                                         former/strided_slice_34[0][0]']  
                                                                                                  
Total params: 4,918
Trainable params: 4,918
Non-trainable params: 0
__________________________________________________________________________________________________
None
Getting valid sampling locations.
Getting locations for .AEX
Getting locations for .AORD
Getting locations for .BFX
Getting locations f

## Calculating Mean Absolute Error ( MAE )

In [4]:
from sklearn.metrics import mean_absolute_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = mean_absolute_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = mean_absolute_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = mean_absolute_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

    
print("*"*100)
print("*"*100)
print("Mean Absolute Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Absolute Error p10 Loss": str(p10_loss.mean()),
    "Mean Absolute Error p50 Loss": str(p50_loss.mean()),
    "Mean Absolute Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:37:26.419390
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

## Calculating Mean Bias Error (MBE)

In [5]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = np.mean(extract_numerical_data(p10_forecast)-extract_numerical_data(targets))
    p50_loss = np.mean(extract_numerical_data(p50_forecast)-extract_numerical_data(targets))
    p90_loss = np.mean(extract_numerical_data(p90_forecast)-extract_numerical_data(targets))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Mean Bias Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Bias Error p10 Loss": str(p10_loss.mean()),
    "Mean Bias Error p50 Loss": str(p50_loss.mean()),
    "Mean Bias Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:38:10.174755
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

## Calculating Relative Absolute Error (RAE)

In [6]:
def rae(actual, predicted):
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = rae(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = rae(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = rae(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Relative Absolute Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Relative Absolute Error p10 Loss": str(p10_loss.mean()),
    "Relative Absolute Error p50 Loss": str(p50_loss.mean()),
    "Relative Absolute Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:38:54.813310
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

## Calculating Mean Absolute Percentage Error ( MAPE )

In [7]:
from sklearn.metrics import mean_absolute_percentage_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = mean_absolute_percentage_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = mean_absolute_percentage_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = mean_absolute_percentage_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Mean Absolute Percentage Error for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Absolute Percentage Error p10 Loss": str(p10_loss.mean()),
    "Mean Absolute Percentage Error p50 Loss": str(p50_loss.mean()),
    "Mean Absolute Percentage Error p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:39:40.967703
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

## Calculating R2 Score

In [8]:
from sklearn.metrics import r2_score

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = r2_score(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = r2_score(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = r2_score(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("R2 Error for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "R2 Error p10 Loss": str(p10_loss.mean()),
    "R2 Error p50 Loss": str(p50_loss.mean()),
    "R2 Error p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:40:26.768245
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

## Calculating Mean Squared Error (MSE)

In [9]:
from sklearn.metrics import mean_squared_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Mean Squared Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Squared Error p10 Loss": str(p10_loss.mean()),
    "Mean Squared Error p50 Loss": str(p50_loss.mean()),
    "Mean Squared Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:41:12.379803
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

## Calculating Root Mean Squared Error (RMSE)

In [10]:
from sklearn.metrics import mean_squared_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    import math
    p10_loss = math.sqrt(mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast)))
    p50_loss = math.sqrt(mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast)))
    p90_loss = math.sqrt(mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast)))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Root Mean Squared Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss, p50_loss, p90_loss))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Root Mean Squared Error p10 Loss": str(p10_loss),
    "Root Mean Squared Error p50 Loss": str(p50_loss),
    "Root Mean Squared Error p90 Loss": str(p90_loss),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:41:58.451604
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

## Calculating Relative Squared Error (RSE)

In [11]:
def relative_squared_error(true, pred):
    true_mean = np.mean(true)
    squared_error_num = np.sum(np.square(true - pred))
    squared_error_den = np.sum(np.square(true - true_mean))
    rse_loss = squared_error_num / squared_error_den
    return rse_loss

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = relative_squared_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = relative_squared_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = relative_squared_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Relative Squared Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Relative Squared Error p10 Loss": str(p10_loss.mean()),
    "Relative Squared Error p50 Loss": str(p50_loss.mean()),
    "Relative Squared Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:42:46.426892
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

## Calculating Mean Squared Logarithmic Error ( MSLE )

In [12]:
"""
from sklearn.metrics import mean_squared_log_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        '''Strips out forecast time and identifier columns.'''
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Mean Squared Log Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Squared Log Error p10 Loss": str(p10_loss.mean()),
    "Mean Squared Log Error p50 Loss": str(p50_loss.mean()),
    "Mean Squared Log Error p90 Loss": str(p90_loss.mean()),
})

"""

'\nfrom sklearn.metrics import mean_squared_log_error\n\nprint("*** Running tests ***")\ntf.reset_default_graph()\n\nwith tf.Graph().as_default(), tf.Session(config=tf_config) as sess:\n\n    tf.keras.backend.set_session(sess)\n    best_params = opt_manager.get_best_params()\n    model = ModelClass(best_params, use_cudnn=use_gpu)\n\n    model.load(opt_manager.hyperparam_folder)\n\n    print("Computing best validation loss")\n    val_loss = model.evaluate(valid)\n\n    print("Computing test loss")\n    output_map = model.predict(test, return_targets=True)\n    targets = data_formatter.format_predictions(output_map["targets"])\n\n    p10_forecast = data_formatter.format_predictions(output_map["p10"])\n    p50_forecast = data_formatter.format_predictions(output_map["p50"])\n    p90_forecast = data_formatter.format_predictions(output_map["p90"])\n\n    def extract_numerical_data(data):\n        \'\'\'Strips out forecast time and identifier columns.\'\'\'\n        return data[[\n           

## Calculating Root Mean Squared Logarithmic Error ( RMSLE )

In [13]:
"""
from sklearn.metrics import mean_squared_log_error
import math 

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        '''Strips out forecast time and identifier columns.'''
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = math.sqrt(mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast)))
    p50_loss = math.sqrt(mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast)))
    p90_loss = math.sqrt(mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast)))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Root Mean Squared Log Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Root Mean Squared Log Error p10 Loss": str(p10_loss.mean()),
    "Root Mean Squared Log Error p50 Loss": str(p50_loss.mean()),
    "Root Mean Squared Log Error p90 Loss": str(p90_loss.mean()),
})
"""


'\nfrom sklearn.metrics import mean_squared_log_error\nimport math \n\nprint("*** Running tests ***")\ntf.reset_default_graph()\n\nwith tf.Graph().as_default(), tf.Session(config=tf_config) as sess:\n\n    tf.keras.backend.set_session(sess)\n    best_params = opt_manager.get_best_params()\n    model = ModelClass(best_params, use_cudnn=use_gpu)\n\n    model.load(opt_manager.hyperparam_folder)\n\n    print("Computing best validation loss")\n    val_loss = model.evaluate(valid)\n\n    print("Computing test loss")\n    output_map = model.predict(test, return_targets=True)\n    targets = data_formatter.format_predictions(output_map["targets"])\n\n    p10_forecast = data_formatter.format_predictions(output_map["p10"])\n    p50_forecast = data_formatter.format_predictions(output_map["p50"])\n    p90_forecast = data_formatter.format_predictions(output_map["p90"])\n\n    def extract_numerical_data(data):\n        \'\'\'Strips out forecast time and identifier columns.\'\'\'\n        return data[

## Calculating Huber Loss, delta = 0.5

In [14]:
def huber_loss(true, pred, delta):
    huber_mse = 0.5 * np.square(true - pred)
    huber_mae = delta * (np.abs(true - pred) - 0.5 * (np.square(delta)))
    return np.where(np.abs(true - pred) <= delta, huber_mse, huber_mae)


print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),0.5)
    p50_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),0.5)
    p90_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),0.5)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Huber Loss with Delta 0.5 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Huber Loss Delta with-0.5 p10 Loss": str(p10_loss.mean()),
    "Huber Loss Delta with-0.5 p50 Loss": str(p50_loss.mean()),
    "Huber Loss Delta with-0.5 p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:43:35.209346
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

## Calculating Huber Loss, delta = 1

In [15]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),1)
    p50_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),1)
    p90_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),1)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Huber Loss with Delta 1.0 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Huber Loss Delta with-1.0 p10 Loss": str(p10_loss.mean()),
    "Huber Loss Delta with-1.0 p50 Loss": str(p50_loss.mean()),
    "Huber Loss Delta with-1.0 p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:44:24.470309
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

## Calculating Huber Loss, delta = 5

In [16]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),5)
    p50_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),5)
    p90_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),5)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Huber Loss with Delta 5.0 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Huber Loss Delta with-5.0 p10 Loss": str(p10_loss.mean()),
    "Huber Loss Delta with-5.0 p50 Loss": str(p50_loss.mean()),
    "Huber Loss Delta with-5.0 p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:45:13.959024
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

## Calculating Log Cosh Loss

In [17]:
def log_cosh(true, pred):
    logcosh = np.log(np.cosh(pred - true))
    logcosh_loss = np.sum(logcosh)
    return logcosh_loss


print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = log_cosh(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = log_cosh(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = log_cosh(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Log Cosh Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Log Cosh Loss p10 Loss": str(p10_loss.mean()),
    "Log Cosh Loss p50 Loss": str(p50_loss.mean()),
    "Log Cosh Loss p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:46:04.389638
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

## Implementing Quantile Loss , gamma=0.25

In [18]:
def quantile_loss(true, pred, gamma):
    val1 = gamma * np.abs(true - pred)
    val2 = (1-gamma) * np.abs(true - pred)
    q_loss = np.where(true >= pred, val1, val2)
    return q_loss


print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),0.25)
    p50_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),0.25)
    p90_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),0.25)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Quantile Loss with Gamma 0.25 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Quantile Loss Gamma with-0.25 p10 Loss": str(p10_loss.mean()),
    "Quantile Loss Gamma with-0.25 p50 Loss": str(p50_loss.mean()),
    "Quantile Loss Gamma with-0.25 p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:46:55.054222
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

## Implementing Quantile Loss , gamma=0.5

In [19]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),0.50)
    p50_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),0.50)
    p90_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),0.50)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Quantile Loss with Gamma 0.5 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Quantile Loss Gamma with-0.5 p10 Loss": str(p10_loss.mean()),
    "Quantile Loss Gamma with-0.5 p50 Loss": str(p50_loss.mean()),
    "Quantile Loss Gamma with-0.5 p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:47:46.234283
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

## Implementing Quantile Loss , gamma=0.75

In [20]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),0.75)
    p50_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),0.75)
    p90_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),0.75)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Quantile Loss with Gamma 0.75 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Quantile Loss Gamma with-0.75 p10 Loss": str(p10_loss.mean()),
    "Quantile Loss Gamma with-0.75 p50 Loss": str(p50_loss.mean()),
    "Quantile Loss Gamma with-0.75 p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_dataset/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOW

                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_15[0][0]',  
                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_lay

 sequential_3 (Sequential)      (None, 257, 5)       60          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_5[0][0]']   
                                                                                                  
 time_distributed (TimeDistribu  (None, 257, 5)      10          ['tf_op_layer_TemporalFusionTrans
 ted)                                                            former/strided_slice_8[0][0]']   
                                                                                                  
 dense_14 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 dense_15 (Dense)               (None, 5)            30          ['dropout_2[0][0]']              
                                                                                                  
 tf_op_lay

 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_17 (Tens                                  former/concat[0][0]']            
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 252, 5)]    0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_18 (Tens                                  former/concat[0][0]']            
 orFlowOpL

 buted)                                                                                           
                                                                                                  
 activation_9 (Activation)      (None, 252, 5)       0           ['time_distributed_9[0][0]']     
                                                                                                  
 activation_10 (Activation)     (None, 252, 5)       0           ['time_distributed_13[0][0]']    
                                                                                                  
 activation_11 (Activation)     (None, 252, 5)       0           ['time_distributed_17[0][0]']    
                                                                                                  
 activation_12 (Activation)     (None, 252, 5)       0           ['time_distributed_21[0][0]']    
                                                                                                  
 activatio

...

 rmalization)                                                                                     
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5)           10          ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multiply_22 (Multiply)         (None, 5, 5, 5)      0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/ExpandDims_4[0][0]',      
                                                                  'tf_op_layer_TemporalFusionTrans
                                                                 former/stack_5[0][0]']           
                                                                                                  
 dense_16 

 ibuted)                                                         former/ExpandDims_5[0][0]']      
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 257, 5)]    0           ['time_distributed_65[0][0]',    
 sformer/add_21 (TensorFlowOpLa                                   'time_distributed_66[0][0]']    
 yer)                                                                                             
                                                                                                  
 activation_23 (Activation)     (None, 257, 5)       0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/add_21[0][0]']            
                                                                                                  
 time_distributed_67 (TimeDistr  (None, 257, 5)      30          ['activation_23[0][0]']          
 ibuted)  

                                                                  'lambda_1[0][0]']               
                                                                                                  
 activation_24 (Activation)     (None, 257, 257)     0           ['add_22[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 257, 257)     0           ['activation_24[0][0]']          
                                                                                                  
 dense_95 (Dense)               (None, 257, 5)       25          ['layer_normalization_21[0][0]'] 
                                                                                                  
 lambda_2 (Lambda)              (None, 257, 5)       0           ['dropout_22[0][0]',             
                                                                  'dense_95[0][0]']               
          

None
Loading model from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
tensor: TemporalFusionTransformer/cu_dnnlstm/bias (float32) [40]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.00823138  0.01975345 -0.01874785 -0.00564956
 -0.01847196  0.98369956  0.9946649   0.98072773  0.9945151   0.9814217
  0.01985468  0.01998466  0.0058541  -0.01990397 -0.01948459 -0.01689216
  0.01738305 -0.01868449 -0.00586672 -0.01829694]
tensor: TemporalFusionTransformer/cu_dnnlstm/kernel (float32) [5, 20]
[[-0.19362448  0.2999755   0.16169433 -0.46355438  0.29267302 -0.11436983
  -0.18850417  0.4918829   0.4362579   0.05894934 -0.30950692 -0.3545085
  -0.3403069   0.02008956  0.07641476  0.02815538 -0.01188646 -0.35307455
  -0.12934797 -0.02710922]
 [ 0.06194233 -0.27189884 -0.29590264  0.47165683 

INFO:tensorflow:Restoring parameters from volatility_dataset/saved_models/volatility/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:48:37.212020
Best validation loss = 1.562695984022244
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.01
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  volatility_dataset/saved_models/volatility/fixed
num_heads  =  1
stack_size  =  1
total_time_steps  =  257
num_encoder_steps  =  252
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataT

### Saving Experiments on a json file

In [21]:
with open("volatility_dataset_experiments.json", "w") as outfile:
    json.dump(loss_experiment_tracker, outfile)